In [1]:
#Assignment 2 - Project Code Start - Chanakya Thunuguntla - cxt162230
#Importing Packages
import os
import zipfile
import sqlite3
import glob
import csv
from urllib.request import urlretrieve
import pandas as pd
from pandas import ExcelWriter

#Storing new and old directories
olddir=os.getcwd()
print("Creating a staging directory in the location :",os.getcwd())
os.makedirs('staging')
os.chdir(os.getcwd()+'/staging') 
print("The current working directory is:", os.getcwd())

#url for the accessing the medicare data
print("Accessing the medicare governement website for the files......")
link='https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1?content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip'
#Accessing the url
urlretrieve(link,"Hospital_Revised_Flatfile.zip")
#Zip file name and path
zip_ref = zipfile.ZipFile("Hospital_Revised_Flatfile.zip","r")
print("Extraction started...")
#Extracting the zip file
zip_ref.extractall()
zip_ref.close() 
print("Extraction complete")

files=glob.glob("*.csv")

#Function to convert cp1252 to utf-8 format
print("Encoding in progress, please wait....")
for file in files:
    infile=open(file,"rt",encoding='cp1252')
    inputdata=infile.read()
    infile.close()
    outfile=open(file,"wt",encoding='utf-8')
    for a in inputdata:
        if a !='\0':
            outfile.write(a)
    outfile.close()
print("Encoding and decoding is complete")

#function which renames the files 
def renaming_table(glob_dir): 
    Table_Name = glob_dir[:-4]
    print("The file name before conversion is :", Table_Name)
    Table_Name = Table_Name.lower()
    Table_Name = Table_Name.replace(' ', '_')
    Table_Name = Table_Name.replace('-', '_')
    Table_Name = Table_Name.replace('%', 'pct')
    Table_Name = Table_Name.replace('/', '_')
    if(Table_Name[0].islower() == False):
        T = 't_'+Table_Name
        print("Table Name after the conversion is :", T)
        return T;
    else:
        T = Table_Name
        print("Table Name after the conversion is :", T)
        return T;
    
#function which renames the columns  
def renaming_col(C): 
    Column_Name = C
    Column_Name = Column_Name.lower()
    Column_Name = Column_Name.replace(' ', '_')
    Column_Name = Column_Name.replace('-', '_')
    Column_Name = Column_Name.replace('%', 'pct')
    Column_Name = Column_Name.replace('/', '_')
    if(Column_Name[0].islower() == False):
        C = ['c_'+Column_Name]
        return C;
    else:
        C = [Column_Name]
        return C;

delname = "FY2015_Percent_Change_in_Medicare_Payments.csv"
if(os.path.exists(delname)):
    os.remove(delname)
    print("********The file - " , delname + " is found and deleted")

print("Starting to create tables from the files extracted.....")
print ("-----------------------------")

#Change the column names and inserting the data to sqlite
for file_name in glob.glob("*.csv"):
        with open(file_name, 'r', encoding = 'UTF-8') as file:
            reader = csv.reader(file)
            Col_Name = next(reader)
            Tab = renaming_table(file_name)
            Col = []
            for C in Col_Name:
                Col = Col+renaming_col(C)
            print ("The Columns of the above table are:", Col)
            print ("-----------------------------")
            #Defining the connection to db
            conn = sqlite3.connect("medicare_hospital_compare.db")
            #Defining the table
            DDL = "create table if not exists " + Tab + " ("
            for C_Name in Col[:-1]:
                DDL = DDL+C_Name+" text, "
            DDL = DDL+Col[-1]+" text)"
            c1 = conn.cursor()
            #Executing query to create stage  tables
            c1.execute(DDL)
            data_lst_lst = [row for row in reader if row != [' ']]
            data_lst_tup = [tuple(l) for  l in data_lst_lst]
            DML_I = "Insert into " + Tab + " values ("
            i = 1
            while (i<len(Col)):
                DML_I = DML_I + "?,"
                i += 1
            DML_I = DML_I + "?)"
            c2 = conn.cursor()
            c2.executemany(DML_I, data_lst_tup)
            #Commiting to the db
            conn.commit()
        print ("-----------------------------")
        
print("Data insertion to DB is complete")
conn = sqlite3.connect("medicare_hospital_compare.db")
c1 = conn.cursor()
sql_str = "SELECT name FROM sqlite_master WHERE type='table';"
rows = c1.execute(sql_str)
print("The created tables are..")
for row in rows:
    print(row)

#################################################
#Creating the Measures Statistical Analysis MS Excel Workbook
#fetching data from database
#getting values and converting them into excel sheets
#extracting data from the url
print("Getting values and converting them into excel sheets")
url=pd.ExcelFile('http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx',dtype=str)
states=url.parse('Focus States')
states=dict(zip(states['State Name'],states['State Abbreviation']))

#changing the value to keys and keys to value
values=states.keys()
keys=states.values()
statelist_states=dict(zip(keys,values))
#Define the variable with excel creation function. 
#This will be used later when the sheets are ready
writer = ExcelWriter('measures_statistics.xlsx',engine='xlsxwriter')

#Read the file timely and effective care hospital file to create Measures Statistical Analysis MS Excel Workbook
stat=pd.read_csv('Timely and Effective Care - Hospital.csv', index_col = False)

#query out the state, measure_id, measure_name, and score.
stat=stat[['Measure ID','Measure Name','State','Score']]

#remove rows with scores which are Not Available
stat=stat[stat['Score']!='Not Available']

#verify and filter numeric Score values
stat=stat[stat['Score'].apply(lambda x: x.isnumeric())]

#change Score column datatype to int
stat['Score']=stat['Score'].astype(int)

#Using pandas function to sort values by Measure ID
#Group the data by Measure ID and Measure Name 
#Find the minimum, maximum, mean and standard deviation for that measure for all hospitals nationwide.
stat["Minimum"]=stat.sort_values('Measure ID').groupby(['Measure ID','Measure Name'])['Score'].transform('min')
stat["Maximum"]=stat.sort_values('Measure ID').groupby(['Measure ID','Measure Name'])['Score'].transform('max')
stat["Average"]=stat.sort_values('Measure ID').groupby(['Measure ID','Measure Name'])['Score'].transform('mean')
stat["Standard Deviation"]=stat.sort_values('Measure ID').groupby(['Measure ID','Measure Name'])['Score'].transform('std')

#Drop duplicate and header rows
stat=stat.drop_duplicates('Measure ID')
stat= stat.drop(['State','Score'], 1)
stat=stat.sort_values('Measure ID')
#Nationwide sheet is complete so write it to the excel with the sheet name as Nationwide
stat.to_excel(writer,'Nationwide',index=False,encoding='utf-8')

#Now create individual excel sheets with state names.
#Keys now has all the focus states names
for state in keys:
    #As per the requirement all filtering options applied to Nationwide are applied here to state sheets
    stat1=pd.read_csv('Timely and Effective Care - Hospital.csv', index_col = False)
    stat1=stat1[['Measure ID','Measure Name','State','Score']]
    stat1=stat1[stat1['Score']!='Not Available']
    stat1=stat1[stat1['Score'].apply(lambda x: x.isnumeric())]
    stat1['Score']=stat1['Score'].astype(int)
    stat1=stat1[stat1['State']==state]
    #Calcualting statistics for that measure only for hospitals located in that state
    stat1["Minimum"]=stat1.sort_values('Measure ID').groupby(['Measure ID','Measure Name'])['Score'].transform('min')
    stat1["Maximum"]=stat1.sort_values('Measure ID').groupby(['Measure ID','Measure Name'])['Score'].transform('max')
    stat1["Average"]=stat1.sort_values('Measure ID').groupby(['Measure ID','Measure Name'])['Score'].transform('mean')
    stat1["Standard Deviation"]=stat1.sort_values('Measure ID').groupby(['Measure ID','Measure Name'])['Score'].transform('std')
    stat1=stat1.drop_duplicates('Measure ID')
    stat1= stat1.drop(['State','Score'], 1)
    stat1=stat1.sort_values('Measure ID')
    A1=statelist_states[state]
    #Write the calculated info to individual state sheets
    stat1.to_excel(writer,sheet_name=A1,index=False,encoding='utf-8')
writer.close()
#Save the excel
writer.save()
print("measures_statistics.xlsx file is created")
#measures_statistics.xlsx file created
#################################################

#################################################
#Creating the Hospital Ranking MS Excel Workbook
#getting values and converting them into excel sheets
url=pd.ExcelFile('http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx',dtype=str)
#Save the website excel data date to a dataframe
web_rank=pd.read_excel(url,dtype=str, index_col = False)
#Save the info from hospital general information file for rankings
csv_rank=pd.read_csv('Hospital General Information.csv',dtype=str, index_col = False)

#Merge the website and hospital general information by Left Join (Left table is web rank) 
#with ID as Provider ID in common.
combined_rank=web_rank.merge(csv_rank, left_on='Provider ID', right_on='Provider ID', how='left')

#query out “Provider ID”, “Hospital Name”, “City”, “State”, and “County”
combined_rank=combined_rank[['Provider ID','Hospital Name','City','State','County Name']]

#change the column names as per the requirement
combined_rank.columns =['Provider ID','Hospital Name','City','State','County']


writer1 = ExcelWriter('hospital_ranking.xlsx',engine='xlsxwriter')
#Top 100 hospitals located in that state, ordered by rank
combined_rank.head(100).to_excel(writer1,'Nationwide',index=False,encoding='utf-8')
for rank in keys:
    combined_rank1=combined_rank[combined_rank['State']==rank]
    A2=statelist_states[rank]
    combined_rank1.head(100).to_excel(writer1,sheet_name=A2,index=False,encoding='utf-8')
writer1.close()
writer1.save()
print("hospital_ranking.xlsx file created")
#hospital_ranking.xlsx file created
#################################################
#End Of the project code

Creating a staging directory in the location : /Users/chanz
The current working directory is: /Users/chanz/staging
Accessing the medicare governement website for the files......
Extraction started...
Extraction complete
Encoding in progress, please wait.
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
..
Encoding and decoding is complete
********The file -  FY2015_Percent_Change_in_Medicare_Payments.csv is found and deleted
Starting to create tables from the files extracted.....
-----------------------------
The file name before conversion is : Payment and Value of Care - Hospital
Table Name after the conversion is : payment_and_value_of_care___hospital
The Columns of the above table are: ['provider_id', 'hospital_name', 'address', 'city', 'state', 'zip_code', 'county_name', 'phone_number', 'payment_measure_name', 'payment_measure_id', 'payment_category

-----------------------------
The file name before conversion is : Outpatient Imaging Efficiency - State
Table Name after the conversion is : outpatient_imaging_efficiency___state
The Columns of the above table are: ['state', 'measure_id', 'measure_name', 'score', 'footnote', 'measure_start_date', 'measure_end_date']
-----------------------------
-----------------------------
The file name before conversion is : Medicare Hospital Spending by Claim
Table Name after the conversion is : medicare_hospital_spending_by_claim
The Columns of the above table are: ['hospital_name', 'provider_id', 'state', 'period', 'claim_type', 'avg_spending_per_episode_hospital', 'avg_spending_per_episode_state', 'avg_spending_per_episode_nation', 'percent_of_spending_hospital', 'percent_of_spending_state', 'percent_of_spending_nation', 'start_date', 'end_date']
-----------------------------
-----------------------------
The file name before conversion is : Ambulatory Surgical Measures-Facility
Table Name afte

-----------------------------
The file name before conversion is : Complications - National
Table Name after the conversion is : complications___national
The Columns of the above table are: ['measure_name', 'measure_id', 'national_rate', 'number_of_hospitals_worse', 'number_of_hospitals_same', 'number_of_hospitals_better', 'number_of_hospitals_too_few', 'footnote', 'measure_start_date', 'measure_end_date']
-----------------------------
-----------------------------
The file name before conversion is : Footnote Crosswalk
Table Name after the conversion is : footnote_crosswalk
The Columns of the above table are: ['footnote', 'footnote_text']
-----------------------------
-----------------------------
The file name before conversion is : Outpatient Imaging Efficiency - Hospital
Table Name after the conversion is : outpatient_imaging_efficiency___hospital
The Columns of the above table are: ['provider_id', 'hospital_name', 'address', 'city', 'state', 'zip_code', 'county_name', 'phone_numbe

-----------------------------
The file name before conversion is : hvbp_safety_11_10_2016
Table Name after the conversion is : hvbp_safety_11_10_2016
The Columns of the above table are: ['provider_number', 'hospital_name', 'address', 'city', 'state', 'zip_code', 'county_name', 'psi_90_achievement_threshold', 'psi_90_benchmark', 'psi_90_baseline_rate', 'psi_90_performance_rate', 'psi_90_achievement_points', 'psi_90_improvement_points', 'psi_90_measure_score', 'hai_1_achievement_threshold', 'hai_1_benchmark', 'hai_1_baseline_rate', 'hai_1_performance_rate', 'hai_1_achievement_points', 'hai_1_improvement_points', 'hai_1_measure_score', 'hai_2_achievement_threshold', 'hai_2_benchmark', 'hai_2_baseline_rate', 'hai_2_performance_rate', 'hai_2_achievement_points', 'hai_2_improvement_points', 'hai_2_measure_score', 'combined_ssi_measure_score', 'hai_3_achievement_threshold', 'hai_3_benchmark', 'hai_3_baseline_rate', 'hai_3_performance_rate', 'hai_3_achievement_points', 'hai_3_improvement_point

-----------------------------
The file name before conversion is : READMISSION REDUCTION
Table Name after the conversion is : readmission_reduction
The Columns of the above table are: ['hospital_name', 'provider_number', 'state', 'measure_name', 'number_of_discharges', 'footnote', 'excess_readmission_ratio', 'predicted_readmission_rate', 'expected_readmission_rate', 'number_of_readmissions', 'start_date', 'end_date']
-----------------------------
-----------------------------
The file name before conversion is : hvbp_efficiency_11_10_2016
Table Name after the conversion is : hvbp_efficiency_11_10_2016
The Columns of the above table are: ['provider_number', 'hospital_name', 'address', 'city', 'state', 'zip_code', 'county_name', 'mspb_1_achievement_threshold', 'mspb_1_benchmark', 'mspb_1_baseline_rate', 'mspb_1_performance_rate', 'mspb_1_achievement_points', 'mspb_1_improvement_points', 'mspb_1_measure_score']
-----------------------------
-----------------------------
The file name befo